In [1]:
import paho.mqtt.client as mqtt

# mqtt client for BLE
device_id = 0.0
tof = 0.0
rssi = 0.0 
duration = 0.0

def on_connect_BLE(client, userdata, flags, rc):
  client.subscribe("fromBLE")

def on_message_BLE(client, userdata, msg):
    
    global device_id, tof, rssi, duration
    
    data = msg.payload.decode()
    data = bytes.fromhex(data) 
    data = data.decode("utf-8") 
    data = data.split(":")
    data = data[1].split(",")

    device_id = float(data[0])
    tof = float(data[1])
    rssi = float(data[2])
    duration = float(data[3])

    client.disconnect()
    
BLEclient = mqtt.Client("P2") 
BLEclient.connect("localhost",1883,60)

BLEclient.on_connect = on_connect_BLE
BLEclient.on_message = on_message_BLE

In [2]:
import time

startTime = 0.0

#mqtt client for smart socket
socketData = 0.0
socketLabel = 0

def on_connect_socket(client, userdata, flags, rc):
  client.subscribe("fromSocket")

def on_message_socket(client, userdata, msg):
    global socketData, socketLabel, startTime

    newSocketData = float(msg.payload.decode())
    
    if(socketData<10 and newSocketData>10):
        socketLabel = 1
        socketData = newSocketData
        startTime = time.time()
    elif(socketData>10 and newSocketData<10):
        socketLabel = 1
        socketData = newSocketData
        startTime = time.time()
    else:    
        socketLabel = 0
        socketData = newSocketData   
    client.disconnect()

SocketClient = mqtt.Client("P1") 
SocketClient.connect("localhost",1883,60)

SocketClient.on_connect = on_connect_socket
SocketClient.on_message = on_message_socket

In [3]:
import pandas as pd
from sklearn.externals.joblib import load    

scaler=load('std_scaler.bin')
uptickDuration = 10

def getFeature():
    BLEclient.connect("localhost",1883,60)
    BLEclient.loop_forever()

    feature = [[device_id,tof,rssi,duration]]
    df = pd.DataFrame(feature, columns = ['Device ID', 'ToF','RSSI','Duration'])

    global scaler 

    return scaler.transform(df)

def getLabelFromSocket():
    global label, startTime, uptickDuration
    
    if(label==1):
        if((time.time()-startTime)<uptickDuration):
            return label

    SocketClient.connect("localhost",1883,60)
    SocketClient.loop_forever()
    
    return label  

/home/rabbi/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
class Environment1:
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False

        #print(type(self.data))
        return self.data[self.t]
    
    def step(self, act):
        reward = 0
        
        # act = 0: unoccupied, 1: occupied
        if act == self.label[self.t]:
            reward +=1 
        else:
            reward -=1
  
        # set next time
        self.t += 1
        
        return self.data[self.t], reward, self.done # obs, reward, done

In [5]:
from chainer import serializers
import time
import copy
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
import numpy as np


class Q_Network(chainer.Chain):

        def __init__(self, input_size, hidden_size, output_size):
            super(Q_Network, self).__init__(
                fc1 = L.Linear(input_size, hidden_size),
                fc2 = L.Linear(hidden_size, hidden_size),
                fc3 = L.Linear(hidden_size, output_size)
            )

        def __call__(self, x):
            h = F.relu(self.fc1(x))
            h = F.relu(self.fc2(h))
            y = self.fc3(h)
            return y

        def reset(self):
            self.zerograds()

In [6]:
# update Q network online
def updateQ(Q, env):
    Q_ast = copy.deepcopy(Q)
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(Q)

    epoch_num = 10
    step_max = len(env.data)-1
    memory_size = 80
    batch_size = 20
    epsilon = 1.0
    epsilon_decrease = 1e-3
    epsilon_min = 0.1
    start_reduce_epsilon = 20
    train_freq = 10
    update_q_freq = 20
    gamma = 0.97
    show_log_freq = 5

    memory = []
    total_step = 0
    total_rewards = []
    total_losses = []

    correct=0
    
    start = time.time()
    for epoch in range(epoch_num):

        pobs = env.reset()
        step = 0
        done = False
        total_reward = 0
        total_loss = 0

        while not done and step < step_max:
            
            # select act
            pact = np.random.randint(2)
            if np.random.rand() > epsilon:
                pact = Q(np.array(pobs, dtype=np.float32).reshape(1, -1))
                pact = np.argmax(pact.data)

            # act
            obs, reward, done = env.step(pact)
            if(reward>0):
                correct = correct+1
                
            # add memory
            memory.append((pobs, pact, reward, obs, done))
            if len(memory) > memory_size:
                memory.pop(0)   
            
            # train or update q
            if len(memory) == memory_size:
                if total_step % train_freq == 0:
                    shuffled_memory = np.random.permutation(memory)
                    memory_idx = range(len(shuffled_memory))
                    for i in memory_idx[::batch_size]:
                        batch = np.array(shuffled_memory[i:i+batch_size])
                        
                        b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                        b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                        
                        b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                        b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                        b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)
                        q = Q(b_pobs)
                        maxq = np.max(Q_ast(b_obs).data, axis=1)
                        target = copy.deepcopy(q.data)
                        for j in range(batch_size):
                            target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                        Q.reset()
                        loss = F.mean_squared_error(q, target)
                        total_loss += loss.data
                        loss.backward()
                        optimizer.update()

                if total_step % update_q_freq == 0:
                    Q_ast = copy.deepcopy(Q)

            # epsilon
            if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                epsilon -= epsilon_decrease

            # next step
            total_reward += reward
            pobs = obs
            step += 1
            total_step += 1

        total_rewards.append(total_reward)
        total_losses.append(total_loss)

        if (epoch+1) % show_log_freq == 0:
            log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
            log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
            elapsed_time = time.time()-start
            #print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
            start = time.time()
            
        avg_loss = sum(total_losses)/len(total_losses) 
        avg_reward = sum(total_rewards)/len(total_rewards)
        avg_accuracy = correct/(epoch_num*len(env.data))
        
    return Q, avg_loss, avg_reward, avg_accuracy

In [7]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import os.path
from os import path
import itertools

consecutive_signals = 3
feature_per_signal = 4
ble_connection_establish = True

writer = SummaryWriter()

data = []
labels = []
plot_idx = 0

Q = Q_Network(input_size=consecutive_signals*feature_per_signal, hidden_size=100, output_size=2)

if(path.exists("Q.model")):
    print("Loaded Q model")
    serializers.load_npz('Q.model', Q)

print('Hit i twice to give a new label.')

currentLabel=-1

def startTraining():
    try:
        global writer, currentLabel, data, labels, Q, plot_idx, ble_connection_establish
    
        update_Q_at = 100
        dump_memory_at = 10000
        recving_feature = False
        
        received_feature = 0
        while 1 == 1 :

            label = currentLabel

            if(label!= 0 and label!= 1):
                continue

            feature = []    
            
            for i in range(consecutive_signals):    
                feature.append(getFeature())
            
            if(ble_connection_establish):
                print("Connection established. Receiving data.")
                ble_connection_establish = False
            
            feature = np.concatenate(feature).ravel()
            data.append(feature)
            
            received_feature+=1
            if(received_feature%20==0):
                print("Received", received_feature,"/",update_Q_at,"data")
            
            labels.append(label)

            if(len(data)%update_Q_at==0):
                received_feature = 0
                print("Updating Q...")
                Q, avg_loss, avg_reward, avg_accuracy = updateQ(Q, Environment1(data,labels))

                writer.add_scalar('Avg. Loss', avg_loss, plot_idx)
                writer.add_scalar('Avg. Reward', avg_reward, plot_idx)
                writer.add_scalar('Avg. Accuracy', avg_accuracy, plot_idx)
                plot_idx +=1 

                print("Updated. Accuracy: ", avg_accuracy)
                

                with open("accuracy.txt", "a") as myfile:
                    myfile.write(str(avg_accuracy)+"\n")

            if(len(data)==dump_memory_at):
                print("Dumping memory...")
                data = []
                my_label = []
                print("Memory dumped")

    except KeyboardInterrupt:
        serializers.save_npz('Q.model',Q)
        print("Saved the Q model")
        ble_connection_establish = True    
        newLabel = input()
        
        try:
            currentLabel = int(newLabel)            
            print("New label is given: ", newLabel)
            if(currentLabel!=0 and currentLabel!=1):
                print("Updating paused. Label must be 0 or 1 to start the model update.")
            else:
                print("Trying to establish connection with BLE device")
            startTraining()
        
        except ValueError:
            print("Invalid input. Updating paused.\nGive 0 for unocciped room, 1 for occupied room")
            currentLabel = -1
            startTraining()

startTraining()

/home/rabbi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/rabbi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/rabbi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/rabbi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,))

Loaded Q model
Hit i twice to give a new label.
Saved the Q model


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-7-54b7189c9e78>", line 40, in startTraining
    if(label!= 0 and label!= 1):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rabbi/.local/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 885, in _input_request
    ident, reply = self.session.recv(self.stdin_socket, 0)
  File "/home/rabbi/.local/lib/python3.6/site-packages/jupyter_client/session.py", line 803, in recv
    msg_list = socket.recv_multipart(mode, copy=copy)
  File "/home/rabbi/.local/lib/python3.6/site-packages/zmq/sugar/socket.py", line 475, in recv_multipart
    parts = [self.recv(flags, copy=copy, track=track)]
  File "zmq/backend/cython/socket.pyx", line 791, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 827, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 186, in zmq.backend.cython.socke

TypeError: must be str, not list